# Supervised Machine Learning with PySpark

We will be using the Orange Telecom Churn Dataset for this tutorial. This is a relatively well-known data set describing customer turnover in the telecommunications industry. The data we use are being hosted at [https://www.sgi.com/tech/mlc/db/churn.all](https://www.sgi.com/tech/mlc/db/churn.all).

In [ ]:
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests, os, sys

if sys.version_info.major == 3:
    from io import StringIO
else:
    from StringIO import StringIO

import pyspark

warnings.filterwarnings('ignore')

%pylab inline
%config InlineBackend.figure_formats = ['retina']

## Connect to the Spark server

We will be using the SparkSession (`spark`) to access our spark cluster..

In [ ]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

## Obtain the data

The data was originally obtained from https://www.sgi.com/tech/mlc/db/churn.all, but has been made available to us in the `data` directory

In [ ]:
fullpath = 'data/churn.txt'
    
# first line of file
! head -n 5 {fullpath}

As can be seen above, the data do not contain column headings so we will have to add those. 

When we import the data below, the schema (which is similar to a schema in SQL or dtypes in Pandas) has been inferred based on the data.

In [ ]:
# read.csv is very similar to the Pandas version
data = spark.read.csv(fullpath,
                     sep=',',
                     inferSchema=True,
                     header=True)

data.printSchema()

Since there are no column headers, we can see the spark gives default names to all the columns. We will want to change that.

In [ ]:
columns = ["state", "account_length", "area_code", "phone_number", "intl_plan", 
           "voice_mail_plan", "number_vmail_messages", 
           "total_day_minutes", "total_day_calls", "total_day_charge", 
           "total_eve_minutes", "total_eve_calls", "total_eve_charge", 
           "total_night_minutes", "total_night_calls", "total_night_charge", 
           "total_intl_minutes", "total_intl_calls", "total_intl_charge", 
           "number_customer_service_calls", "churned"]

old_columns = data.schema.names

# renaming columns is a little different with PySpark
for old,new in zip(old_columns,columns):
    data = data.withColumnRenamed(old, new)

# view dtypes
data.printSchema()

So we inferred the schema and then set the column names above. However, both of these things can also be explicitly created and then provided to the `read.csv` command.

An example of a defined schema is below, where the text is the column name, then the type (string, double, boolean, double, etc.) is entered. The final `True` indicates that null values can exist.

```python
from pyspark.sql.types import StringType, DoubleType, StructType, StructField

schema = StructType([
    StructField("state", StringType(), True), 
    StructField("account_length", DoubleType(), True), 
    StructField("area_code", StringType(), True), 
    ...
    ...
    StructField("churned", StringType(), True)])
```

Now, let's preview the data as a Pandas dataframe.

In [ ]:
# function to select a few rows of data, convert to a Pandas dataframe, and transpose
def preview(df, n=3):
    return pd.DataFrame(df.take(n), columns=df.columns).T

preview(data)

## Data exploration

Spark SQL allows us to explore data using SQL syntax. Let's examing the breakdown of churned customers. To perform SQL queries, the tables have to be registered using the `registerTempTable` method.

In [ ]:
data.registerTempTable('data')

churn_counts = spark.sql(r"""SELECT churned, COUNT(churned) AS total 
                            FROM data
                            GROUP BY churned""")
churn_counts.show()

Above, we can see that there is a period at the end of the True/False values in the churned column. (There is also a space at the beginning, but that is less obvious.) We will clean all of these issues up later.

Let's examine the churn in the states with the most subscribers. First, determine the 10 states with the most subscribers.

In [ ]:
state_customer_counts = spark.sql(r"""SELECT state, COUNT(state) AS total 
                                     FROM data 
                                     GROUP BY state 
                                     ORDER BY total desc 
                                     LIMIT 10""")
state_customer_counts.show()

In [ ]:
state_churn_counts = spark.sql(r"""SELECT state, COUNT(churned) as churned
                                  FROM data
                                  WHERE churned LIKE '%True%'
                                  GROUP BY state""")

state_churn_counts.show(10)

These are actual PySpark dataframes and they can be joined using either PySpark or SQL. First the join using SQL.

In [ ]:
# register the tables with SQL
state_customer_counts.registerTempTable('state_customer_counts')
state_churn_counts.registerTempTable('state_churn_counts')

# join using SQL
combined_state_data = spark.sql(r"""SELECT cust_cts.state, churned, total
                                   FROM state_customer_counts AS cust_cts
                                   INNER JOIN state_churn_counts AS churn_cts
                                   ON cust_cts.state = churn_cts.state""")
combined_state_data.show()

PySpark also allows dataframes to be joined and sorted in a fashion similar to Pandas.

In [ ]:
combined_state_data2 = state_churn_counts.join(state_customer_counts, 
                                               on='state', 
                                               how='inner')

# sort inversely by total column
combined_state_data2 = combined_state_data2.sort(combined_state_data2.total.desc())

combined_state_data2.show()

## Data cleaning

The columns that are supposed to be boolean (`intl_plan`, `voice_mail_plan`, and `churned`) need some cleaning and their types need to be converted from string to boolean.

In [ ]:
from pyspark.sql.functions import udf

# columns not used in analysis
for field in ['state', 'area_code', 'phone_number']:
    data = data.drop(field)

def trim(string):
    return (string
            .strip()                # string columns have spaces due to csv file format
            .replace('.','')        # remove period at end of each line
            .replace('yes','True')  # convert yes/no to True/False
            .replace('no','False'))

# udf converts a function into one that can be applied over
# a dataframe column. This is kind of like Pandas apply/map functionality.
trim = udf(trim)
    
# apply the function to every string column
for dtype in data.dtypes:
    column = dtype[0]
    if dtype[1] == 'string':
        data = data.withColumn(column, trim(data[column]))

# boolean types converted to integers
for column in ['intl_plan', 'voice_mail_plan']:
    data = data.withColumn(column, data[column].cast('boolean').cast('int'))
    
# predictors have to be of type double for many of Spark's ML models
data = data.withColumn('churned', data['churned'].cast('boolean').cast('double'))

Now that we've cleaned the data, let's make correlation plots of variables.

In [ ]:
sns.set_palette('dark')
sns.set_context('notebook')
sns.set_style('white')

# get the names of columns that are integers or doubles
numeric_features = [t[0] for t in data.dtypes if t[1] in ['int', 'double']]

# sample 10% of this data and convert to a Pandas dataframe
sampled_data = data.select(numeric_features).sample(False, 0.1).toPandas()

# make the scatter plot
axs = pd.scatter_matrix(sampled_data, figsize=(12, 12));

# Rotate axis labels and remove axis ticks
n = len(sampled_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

As we can see from above, there are four pairs of variables that are highly correlated with each other, corresponding to charge and minutes for day, night, evening, and international. This makes sense since the charge like reflects a per-minute rate. We will remove the charge column for each of these.

In [ ]:
# remove one of each pair of highly correlated columns
for field in ['total_day_charge', 'total_night_charge',
              'total_eve_charge', 'total_intl_charge']:
    data = data.drop(field)
    
preview(data)

## Split data

Before modelling, the data are split into train and test data sets. 

We could also do a three-way split for train, validation, and test data using the `randomSplit` method:

```python
train, valid, test = data.randomSplit([0.6, 0.2, 0.2], seed=42)
```

Finally, it is also possible to do a stratified split using the `sampleBy` method of the dataframe.

In [ ]:
train, test = data.randomSplit([0.7, 0.3], seed=42)

## Combine feature columns

Inputs to Scikit-learn models have the data split into X-arrays/dataframes (features) and Y-arrays/dataframes (predictors). PySpark expects both the features and the predictor to be in a single dataframe. However, all the features have to be combined into a single column containing a list of the feature values. To accomplish this, we use [`VectorAssembler`](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler).

Note that VectorAssembler uses transform. This is just like the second step of Scikit-learn's feature transformers for normalization, etc.

We will also be using [`MinMaxScaler`](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.MinMaxScaler) to scale the features for the logistic regression model. (Note that scaling isn't strictly required with logistic regression, but it doesn't hurt and it's good to get experience with how these feature modifications work.

In the cells below, use VectorAssembler to create a packed feature column and then use MinMaxScaler to scale this packed column.

In [ ]:
from pyspark.ml.feature import VectorAssembler

# the feature columns
features = data.schema.names[:-1]

assembler = VectorAssembler(inputCols=features, outputCol='features')

train_pack = assembler.transform(train)
test_pack = assembler.transform(test)

# (optional) remove the columns packed into the feature vector
for field in features:
    train_pack = train_pack.drop(field)
    test_pack  = test_pack.drop(field)

In [ ]:
from pyspark.ml.feature import MinMaxScaler

minmaxscale = MinMaxScaler(inputCol='features', outputCol='features_scaled')
minmaxscale = minmaxscale.fit(train_pack)

train_pack = minmaxscale.transform(train_pack)
test_pack  = minmaxscale.transform(test_pack)

## Basic model building

Now we get to the fun part--machine learning! 

PySpark has two machine learning libraries:

1. MLlib: older, based on RDDs, more complete, but no additional features are being added as of Spark 2.0
2. ML: the *future*, based on DataFrames, less feature complete than MLlib but this is improving

Often the name MLlib is used to refer to both libraries, so context is important and this can be confusing. We will be using ML whenever possible.

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, \
                                      GBTClassifier, LogisticRegression

We will begin by training a number of models and evaluating the results. Notice that the `fit` method is similar to Scikit-learn. However, `transform` is used with PySpark instead of `predict`. The syntax is similar among different PySpark ML models, though.

Try to train the logistic regression model with the scaled features column. For the other models, just use the unscaled features.

In [ ]:
# logistic regression
lr = LogisticRegression(labelCol='churned', 
                        featuresCol='features_scaled',
                        predictionCol='prediction')

lr_model = lr.fit(train_pack)
lr_pred = lr_model.transform(test_pack)

# Decision Tree (depth = 2)
dt = DecisionTreeClassifier(maxDepth=2, 
                            labelCol='churned', 
                            featuresCol='features',
                            predictionCol='prediction')

dt_model = dt.fit(train_pack)
dt_pred = dt_model.transform(test_pack)

# Random Forest (also of depth = 2)
rf = RandomForestClassifier(maxDepth=2, 
                            labelCol='churned',
                            featuresCol='features',
                            predictionCol='prediction')

rf_model = rf.fit(train_pack)
rf_pred = rf_model.transform(test_pack)

# Gradient Boosted Tree (depth = 2)
gbt = GBTClassifier(maxDepth=2, 
                    labelCol='churned', 
                    featuresCol='features',
                    predictionCol='prediction')

gbt_model = gbt.fit(train_pack)
gbt_pred = gbt_model.transform(test_pack)

## Model evaluation

PySpark also has error metrics that are similar to those found in Scikit-learn. They are divided into metrics appropriate only for binary classifications (ROC and Precision-Recall curve related metrics) and those that also apply to multi-class classifications.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, \
                                  MulticlassClassificationEvaluator

accuracy = MulticlassClassificationEvaluator(labelCol='churned', 
                                             predictionCol='prediction',
                                             metricName='accuracy')

precision = MulticlassClassificationEvaluator(labelCol='churned', 
                                              predictionCol='prediction',
                                              metricName='weightedPrecision')

recall = MulticlassClassificationEvaluator(labelCol='churned', 
                                           predictionCol='prediction',
                                           metricName='weightedRecall')

f1 = MulticlassClassificationEvaluator(labelCol='churned', 
                                       predictionCol='prediction',
                                       metricName='f1')

areaROC = BinaryClassificationEvaluator(labelCol='churned',
                                        rawPredictionCol='prediction',
                                        metricName='areaUnderROC')

areaPR = BinaryClassificationEvaluator(labelCol='churned',
                                       rawPredictionCol='prediction',
                                       metricName='areaUnderPR')

These metrics can be used to create a table of the values for each model's predictions.

In [ ]:
# the error metrics
metrics = [accuracy, precision, recall, f1, areaROC, areaPR]
metric_labels = ['accuracy', 'precision', 'recall', 'f1', 'areaROC', 'areaPR']

# the predictions from each model
predictions = [lr_pred, dt_pred, rf_pred, gbt_pred]
predict_labels = ['LR', 'DT', 'RF', 'GBT']

eval_list = list()

# for each model's predictions, calculate error metrics
# and add to a Pandas series
for pred in zip(predict_labels, predictions):
    name = pred[0]
    predict = pred[1]
    
    metric_vals = pd.Series(dict([(x[0], x[1].evaluate(predict)) 
                                 for x in zip(metric_labels, metrics)]),
                            name=name)
    eval_list.append(metric_vals)
    
# combine all the series into a dataframe
eval_df = pd.concat(eval_list, axis=1).T
eval_df = eval_df[metric_labels]
eval_df

Similar metrics (plus a confusion matrix) are also available in MLlib, the older library. We will have to convert the dataframe to an RDD to use these metrics. These metrics have the advantage that they are a little easier (and less verbose) to implement. There is also a confusion matrix function.

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics, \
                                     MulticlassMetrics

# let's make a function this time so it's reuseable

def evaluate_model_predictions(prediction_list, prediction_labels):
    
    eval_list = list()

    # for each model's predictions, calculate error metrics
    # and add to a Pandas series
    for pred in zip(prediction_labels, prediction_list):
        name = pred[0]
        predict = pred[1]

        # here is where the prediction dataframe is converted to an rdd
        predict_rdd = predict[['prediction', 'churned']].rdd
        b_metrics = BinaryClassificationMetrics(predict_rdd)
        m_metrics = MulticlassMetrics(predict_rdd)

        metric_vals = pd.Series({'accuracy'        : m_metrics.accuracy,
                                 'precision'       : m_metrics.precision(0),
                                 'recall'          : m_metrics.recall(0),
                                 'f1'              : m_metrics.fMeasure(),
                                 'areaROC'         : b_metrics.areaUnderROC,
                                 'areaPR'          : b_metrics.areaUnderPR,
                                 'confusion matrix': (m_metrics.confusionMatrix()
                                                      .toArray().astype('int'))},
                                name=name)
        eval_list.append(metric_vals)

    # combine all the series into a dataframe
    eval_df = pd.concat(eval_list, axis=1).T
    
    metric_labels = ['accuracy', 'precision', 'recall', 'f1', 'areaROC', 'areaPR', 'confusion matrix']
    eval_df = eval_df[metric_labels]
    
    return eval_df


# the predictions from each model
predictions = [lr_pred, dt_pred, rf_pred, gbt_pred]
predict_labels = ['LR', 'DT', 'RF', 'GBT']

eval_df2 = evaluate_model_predictions(predictions, predict_labels)
eval_df2

Note that the calculations for precision, recall, and f1 are a little different from the first set. This is because the dataframe calculations use weighted precision and recall.

## Combine into pipeline

Any feature transformations and model fitting can be combined into a pipeline. Pipelines are useful because:

1. The data can be split  *before* any transformations that may lead to leakage of the test data.
2. Simultaneously ensure that the split data sets are treated identically.

We don't do much in this example for feature transformation other than assembling all the features into one column and scaling, but let's use that to create a pipeline for the logistic regression model we used above.

If we *did* do extensive feature engineering, then having a pipeline would be extremely beneificial. (Just like in scikit-learn.)

In [ ]:
from pyspark.ml import Pipeline

# the pipline
pipeline = Pipeline(stages=[assembler, minmaxscale, lr]) 

# fit and transform
lr_model2 = pipeline.fit(train)
lr_pred2 = lr_model2.transform(test)

# the results are the same for GBT as the second table above
evaluate_model_predictions([lr_pred2], ['LR'])

## Grid search and cross validation

PySpark also has gridsearch and crossvalidation functionalities, similar to Scikit-learn's. Let's do a quick example with GradientBoostedTrees.

In [ ]:
# may take ~5min to run this cell
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# the GradientBoostedTree model
gbt2 = GBTClassifier(featuresCol='features',
                     labelCol='churned', 
                     predictionCol='prediction')
 
# the pipline
pipeline = Pipeline(stages=[assembler, gbt2]) 

# the parameter grid--we'll optimize maxDepth and stepSize
paramgrid = (ParamGridBuilder().addGrid(gbt2.maxDepth, [4,6])
                               .addGrid(gbt2.stepSize, [0.001, 0.01, 0.1, 1.0]).build())

# use f1 score as the evaluation metric for best model 
evaluator = MulticlassClassificationEvaluator(labelCol='churned', 
                                              predictionCol='prediction', 
                                              metricName='f1') 

# use 3-fold cross validation 
crossval = CrossValidator(estimator=pipeline, 
                          estimatorParamMaps=paramgrid, 
                          evaluator=evaluator, 
                          numFolds=3) 

gbt2_model = crossval.fit(train) 

Much like with Scikit-learn's gridsearch function, we can probe for the best model and view its attributes. 

First, we have to select the best model, though.

In [ ]:
# return the best pipeline based on f1 score
best_pipeline = gbt2_model.bestModel

# return the best GBT model, which is the second step of the pipeline
best_gbt_model = best_pipeline.stages[1]

Unfortunately, not all model parameters can be extracted with PySpark, but we can extract the individual trees for GradientBoostedTrees and then examine them.

In [ ]:
# how many trees were there?
len(best_gbt_model.trees)

In [ ]:
# list some of the trees
best_gbt_model.trees[:5]

In [ ]:
# what were their weights in the GBT model?
best_gbt_model.treeWeights[:5]

In [ ]:
# what was the best depth?
[x.depth for x in best_gbt_model.trees]

We can also examine feature importances, much like Scikit-learn.

In [ ]:
# extract feature importances
feature_importances = best_gbt_model.featureImportances.toArray()

# extract feature names, except for the predictor
feature_names = train.columns[:-1]

feature_series = (pd.Series(dict(zip(feature_names, feature_importances)))
                  .sort_values(ascending=True))

feature_series

Let's plot them in a bar graph. Total minutes (day, evening, night, or international) are the biggest predictor of churn.

In [ ]:
sns.set_palette('dark')
sns.set_context('notebook')
sns.set_style('white')

ax = feature_series.plot(kind='barh')
_ = ax.set(xlabel='Relative Importance', 
           ylabel='Features', 
           title='Feature Importances for Best GradientBoostedTree Model')

fig = plt.gcf()

# save to the folder shared with the AWS instance
# this is the same folder as the notebook is executed from
# so a path is not required
fig.savefig('img/feature_importances.pdf')

## Measure error on test data set

Let's use the best model to measure the error on the test data set. We used f1 score for the gridsearch metric, and as can be seen below, this improved from 0.916 to 0.943 with the gridsearch.

In [ ]:
gbt_pred_test = best_pipeline.transform(test)

evaluate_model_predictions([gbt_pred_test], ['GBT_GridSearch'])

## Save the model for later use

PySpark trained models can be saved for later use and then reloaded. Conceptually, this is much like pickling, except the output is a directory.

In [ ]:
model_output_name = 'gbt_model_pipeline_crossval'

# models will not overwrite existing ones of the same name
import shutil
if os.path.exists(model_output_name):
    shutil.rmtree(model_output_name)

best_gbt_model.save(model_output_name)

! ls {model_output_name}

Now the model can be loaded here or in other analysis. Note that we have to import the *model* instance to load the file. Previously, we had only imported the classifier.

In [ ]:
from pyspark.ml.classification import GBTClassificationModel

reloaded_model = GBTClassificationModel.load(model_output_name)
reloaded_model